In [1]:
# === Cell 1: Mount Google Drive and set file path ===
from google.colab import drive
# Authenticate and mount
drive.mount('/content/drive')
# Place your artichoke_pizza.txt under /MyDrive/pizza_recipes/
FILE_PATH = '/content/drive/MyDrive/pizza_recipes/artichoke_pizza.txt'
print(f"Using recipe file: {FILE_PATH}")


Mounted at /content/drive
Using recipe file: /content/drive/MyDrive/pizza_recipes/artichoke_pizza.txt


In [2]:
# === Cell 2: Imports & OpenAI client ===
import json, re
from getpass import getpass
from openai import OpenAI
# Enter your API key
token = getpass('Enter OpenAI API key: ')
client = OpenAI(api_key=token)

Enter OpenAI API key: ··········


In [3]:
# === Cell 3: Load recipe JSON ===
with open(FILE_PATH,'r',encoding='utf-8') as f:
    data = json.load(f)
print('Recipe title:', data['title'])
print('Step count:', len(data['table']))

Recipe title: artichoke_pizza
Step count: 4


In [6]:
# === Debug Cell: inspect the first row of the table ===
print(data['table'][0].keys())
# 期待される出力例: dict_keys(['instructions', 'input', 'actions', 'output'])
# もし 'actions' ではなく 'action' なら次のステップで置き換えます。


dict_keys(['instructions', 'input', 'action', 'output'])


In [7]:
# === Cell 4: CoT prompt function ===
def cot_prompt(instruction, action, prev_out='NA'):
    prev = f"Previous output: {prev_out}\n" if prev_out!='NA' else ''
    prompt = (
        f"Instruction: {instruction}\n"
        f"Action: {action}\n"
        f"{prev}Think step by step, then output ONLY:\n"
        "<in>: (item1; item2; ...)</in>\n"
        "<out>: result_phrase</out>"
    )
    resp = client.chat.completions.create(
        model='gpt-4o', messages=[
            {'role':'system','content':'You are a concise cooking assistant.'},
            {'role':'user','content':prompt}
        ], temperature=0.0, max_tokens=150)
    return resp.choices[0].message.content.strip()

In [8]:
# === Cell 5: Reasoning Preparation (CoT) ===
results = []
prev_out = 'NA'
for i, row in enumerate(data['table'], start=1):
    instr = row.get('instructions') or row.get('instruction')
    act   = row.get('actions')     or row.get('action')
    print(f"\n--- Step {i}: {instr} ---")
    out = cot_prompt(instr, act, prev_out)
    print(out)
    results.append({'step':i, 'instr':instr, 'act':act, 'cot':out})
    # 抜き出しも同じまま
    m = re.search(r"<out>\s*(.*?)\s*</out>", out)
    prev_out = m.group(1).strip() if m else 'NA'



--- Step 1: heat up your oven as hot as it gets . ---
<in>: (oven)</in>
<out>: The oven is preheating to its maximum temperature.</out>

--- Step 2: spread tomato_sauce evenly to the both pizzas . ---
<in>: (tomato_sauce; pizza1; pizza2)</in>
<out>: Tomato sauce is spread evenly on both pizzas.</out>

--- Step 3: spread the mushrooms , artichokes and olives on the pizza . ---
<in>: (mushrooms; artichokes; olives)</in>
<out>: Mushrooms, artichokes, and olives are spread evenly on the pizza.</out>

--- Step 4: bake in the oven for about 10 minutes or longer , depending on how hot your oven is . ---
<in>: (pizza; oven)</in>
<out>: The pizza is baked in the oven for about 10 minutes or longer, depending on the oven's temperature.</out>


In [9]:
# === Cell 6: Parse CoT outputs to extract predicted in/out ===
for r in results:
    cot = r['cot']
    inm = re.search(r"<in>\s*(.*?)\s*</in>", cot)
    outm= re.search(r"<out>\s*(.*?)\s*</out>", cot)
    r['pred_in']  = inm.group(1).strip() if inm else 'NA'
    r['pred_out'] = outm.group(1).strip() if outm else 'NA'
print('\nPredicted I/O:')
for r in results: print(r['step'], r['pred_in'], '->', r['pred_out'])


Predicted I/O:
1 : (oven) -> : The oven is preheating to its maximum temperature.
2 : (tomato_sauce; pizza1; pizza2) -> : Tomato sauce is spread evenly on both pizzas.
3 : (mushrooms; artichokes; olives) -> : Mushrooms, artichokes, and olives are spread evenly on the pizza.
4 : (pizza; oven) -> : The pizza is baked in the oven for about 10 minutes or longer, depending on the oven's temperature.


In [10]:

# === Cell 7: Define Cooking Glossary (Ground Truth mapping) ===
glossary = {
    'preheat':    {'in':'NA','out':'NA'},
    'mix':        {'in':'(flour; water)','out':'dough'},
    'spread':     {'in':'(dough; tomato_sauce)','out':'sauced dough'},
    'add_toppings':{'in':'(sauced dough; mushrooms; artichokes; olives)','out':'topped dough'},
    'bake':       {'in':'(topped dough)','out':'baked pizza'}
}


In [11]:
# === Cell 8: Dynamic Knowledge Adapting ===
for r in results:
    action = r['act']
    map_key = action if action in glossary else 'add_toppings' if 'spread' in action else None
    if map_key and glossary[map_key]:
        r['corr_in']  = glossary[map_key]['in']
        r['corr_out'] = glossary[map_key]['out']
    else:
        r['corr_in'], r['corr_out'] = r['pred_in'], r['pred_out']
print('\nCorrected I/O after Knowledge Adapting:')
for r in results: print(r['step'], r['corr_in'], '->', r['corr_out'])


Corrected I/O after Knowledge Adapting:
1 : (oven) -> : The oven is preheating to its maximum temperature.
2 (dough; tomato_sauce) -> sauced dough
3 (dough; tomato_sauce) -> sauced dough
4 (topped dough) -> baked pizza


In [12]:
# === Cell 9: Answer Consolidation ===
import pandas as pd
df = pd.DataFrame([{
    'Step':    r['step'],
    'Instruction': r['instr'],
    'Input':   r['corr_in'],
    'Output':  r['corr_out']
} for r in results])
df

,Step,Instruction,Input,Output
0,1,heat up your oven as hot as it gets .,: (oven),: The oven is preheating to its maximum temper...
1,2,spread tomato_sauce evenly to the both pizzas .,(dough; tomato_sauce),sauced dough
2,3,"spread the mushrooms , artichokes and olives o...",(dough; tomato_sauce),sauced dough
3,4,bake in the oven for about 10 minutes or longe...,(topped dough),baked pizza
